In [1]:
!nvidia-smi

Wed May 17 03:57:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install datasets -qq
!pip install transformers -qq
!pip install rouge_score evaluate nltk -qq

In [3]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
from datasets import load_dataset
import datasets
import evaluate
import nltk
import torch
import pandas as pd 
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from nltk.tokenize import word_tokenize

In [5]:
import tensorflow as tf

In [6]:
model_checkpoint = "gpt2"
from transformers import AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [8]:
from datasets import load_dataset, load_metric

In [9]:
metric = load_metric("rouge")

<ipython-input-9-7154407f1f07>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [10]:
# Load the XSum dataset
raw_datasets = load_dataset('xsum', split="train[:10000]")
raw_datasets = raw_datasets.train_test_split(test_size=0.1)
raw_datasets
raw_datasets["train"][0]

{'document': 'Adrian Pogmore, 51, used the aircraft to film people sunbathing naked and a couple, who were his friends, having sex in their garden.\nFour other men all deny charges of misconduct in a public office.\nGiving evidence at Sheffield Crown Court, a former colleague said he did not know Pogmore was "into voyeurism".\nMore stories from across Yorkshire\nPolice officers Matthew Lucas, 42, and Lee Walls, 47, and helicopter pilots Matthew Loosemore, 45, and Malcolm Reeves, 64, are all on trial.\nPogmore made four recordings from the aircraft between 2007 and 2012, including filming two naturists sitting outside a caravan on a campsite and his friends having sex, the court heard.\nThe jury was told he knew the couple because they "shared his sexual interest in the swinging scene" and the pair had "brazenly put on a show" for the helicopter.\nWhen asked by Mr Loosemore\'s defence barrister, Neil Fitzgibbon, if he believed it was appropriate for someone "into swinging and voyeurism"

In [11]:
tokenizer.pad_token = tokenizer.eos_token

In [12]:
def tokenize_function(examples):
    return tokenizer(
        examples['summary'],
        padding='max_length',
        truncation=True,
        max_length=512,
    )

# Apply the tokenization function to the dataset
tokenized_dataset = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=raw_datasets['train'].column_names
)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
nltk.download("punkt", quiet=True)

True

In [14]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [15]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [16]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [17]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

batch_size = 8

training_args = TrainingArguments(
    output_dir="res",
    num_train_epochs=2,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    weight_decay=0.1,
    evaluation_strategy='steps',
    eval_steps=500,
    warmup_steps=500,
    logging_dir="logs",
    logging_steps=500,
    save_total_limit=2,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)


In [18]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,3.542000,3.235963


TrainOutput(global_step=562, training_loss=3.5229568209936613, metrics={'train_runtime': 2539.9821, 'train_samples_per_second': 7.087, 'train_steps_per_second': 0.221, 'total_flos': 4699075903488000.0, 'train_loss': 3.5229568209936613, 'epoch': 2.0})

In [43]:
def generate_story(prompt, model, tokenizer, max_length=512, num_stories=1):
    for i in range(1, num_stories+1):
        # Tokenize the prompt
        input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
        attention_mask = input_ids.ne(tokenizer.pad_token_id).to(device)

        # Generate the story continuation
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_beams=10,
            early_stopping=True,
            no_repeat_ngram_size=2,
        )

        # Decode the generated output
        generated_story = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Print the generated story with its index
        print(f"Generated {i}: {generated_story}\n")

In [45]:
prompt = "Once upon a time"
generated_story = generate_story(prompt, model, tokenizer, max_length=512, num_stories=3)
print(generated_story)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated 1: Once upon a time in the history of the United States, President George W. Bush and Secretary of State Madeleine Albright decided to send a message to the world.

In a letter to President Barack Obama, the two men wrote, "It is with a heavy heart that I write to you. I want you to know that you are not alone, but we are all together. We are together, and we will continue to be together." The letter was signed by former US President Bill Clinton and former President Jimmy Carter, who is now President Donald Trump's son-in-law and one of his most senior advisers. In the letter, they wrote: "We are united in our determination to make America great again. This is why I am so proud to have you as my President-elect. As you know, I have been a big fan of you and your family for a long time. You are a great friend to me and to all of us. Thank you for your hard work and for all the support you have given me over the years. And I will miss you very much."
The letter is the latest i

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated 2: Once upon a time it seemed as if the world was going to be a better place than it is now. But it was not.

The world has changed dramatically since the end of the Cold War. The world is no longer the same as it once was. There are no more countries in Africa, Asia, Africa and Latin America, and the Middle East and North Africa. In fact, there are only a handful of countries that have changed at least as much as they did in the 1980s and 1990s. As a result of this change, the number of people who live in poor countries has fallen by more than a third over the past 20 years, according to a new report by the International Monetary Fund (IMF). This is the first time the IMF has looked at the scale of change in a country's economic and political life. It found that the share of poor people living in countries with the lowest levels of income inequality has increased in recent decades. This has been especially true in developing countries, where the gap between rich and poor has

In [47]:
prompt = "Today I have a make up class to report the final of deep learning"
generated_story = generate_story(prompt, model, tokenizer, max_length=512, num_stories=3)
print(generated_story)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated 1: Today I have a make up class to report the final of deep learning. In this class I'm going to take a look at some of the topics I've covered so far. I hope you find it interesting and useful.

If you have any questions, feel free to let me know in the comments below.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated 2: Today I have a make up class to report the final of deep learning at the University of Cambridge.

It is the first of its kind in the UK. I am proud to be a part of it, and I hope you will join me on this journey.

Generated 3: Today I have a make up class to report the final of deep learning. In this class, I'm going to show you how Deep Learning can be used as a teaching tool in a classroom.

As you can see in the video, Deep learning is being used in classrooms across the UK. This is the first time I've been able to see how it has been used to teach a new generation of students. I hope you enjoy this video as much as I do. If you have any questions, feel free to leave a comment below and I'll be happy to answer them.

None
